In [198]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

In [295]:
# Get data
data_jokes = 'jokes.csv'
data = 'train.csv'

# Initialize dataframes
df = pd.read_csv(data)
df_jokes = pd.read_csv(data_jokes)

#df.head()

In [296]:
#df_jokes.head()

In [201]:
df.shape

(1092059, 4)

In [297]:
# Merge dataframes on joke id
df = pd.merge(df, df_jokes, left_on='joke_id', right_on='joke_id', how='left').drop('id', axis=1)
#df.head()

In [298]:
# Helper function to display joke text
def joke_text(joke_id):
    joke_text = df.at[joke_id, 'joke_text']
    return joke_text

In [299]:
#joke_text(1)

In [286]:
# If we want to narrow the ratings we can use this cell

# Users per joke
#users_per_joke = df.joke_id.value_counts()
#users_per_joke.shape

# Jokes per users
#jokes_per_user = df.user_id.value_counts()
#jokes_per_user.shape

# Picking jokes that has been rated by more than 10 users
#df = df[df['joke_id'].isin(users_per_joke[users_per_joke>10].index)]

# User who rated more than 10 jokes
#df = df[df['user_id'].isin(jokes_per_user[jokes_per_user>10].index)]

In [300]:
# Create ratings matrix
rating_matrix_pivot = pd.pivot_table(df, values='Rating', index=['user_id'], columns=['joke_id']).fillna(0)

#rating_matrix_pivot.tail()

In [301]:
# Create sparse matrix
from scipy.sparse import csr_matrix
# Create sparse matrix
ratings_matrix = csr_matrix(rating_matrix_pivot.values)
#ratings_matrix

In [303]:
# Implement KNN with cosine metric
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='cosine', algorithm='brute')
model = model.fit(ratings_matrix)

In [306]:
# Helper function to recommend jokes

def recommend_jokes():
    query_index = np.random.choice(rating_matrix_pivot.shape[0])
    distances, indices = model.kneighbors(rating_matrix_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 5)
    
    joke_list = []
    
    user = ''
    
    for i in range(0, len(distances.flatten())):
        if i == 0:
            user_index = str(query_index) 
            user = user_index

        else:
            try:
                joke_list.append(joke_text(rating_matrix_pivot.index[indices.flatten()[i]]))
                
            except:
                joke_list.append("")
    
    user = 'Recommended jokes for user ' + user
        
    df = pd.DataFrame(joke_list, columns = [user])
    df.set_index(user, inplace=True)
    

    return df

In [307]:
# Try function
#recommend_jokes()

# Use the button below to get recommendations for jokes
[GitHub Repo](https://github.com/helenabarmer/joke_recommender)


In [310]:
# Create widget button
button = widgets.Button(description="Generate Joke", button_style='success')

# Output
output = widgets.Output(layout={'border': '1px solid black'})

# Display button
display(button, output)

def joke(b):
       with output:
            clear_output(True)
            display(recommend_jokes())
            
            
button.on_click(joke)

Button(button_style='success', description='Generate Joke', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))